In [18]:
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore") 
import datetime
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [19]:
today=str(datetime.datetime.today().strftime('%d_%m_%Y'))

In [20]:
INC = pd.read_excel(r'C:\NotBackedUp\SOE\source_files\INC\incident.xlsx')

In [21]:
list(INC)

['Number',
 'Active',
 'Created',
 'State',
 'Activity due',
 'Additional assignee list',
 'Approval',
 'Approval history',
 'Approval set',
 'Assigned to',
 'Assignment group',
 'Business duration',
 'Created by',
 'Current Domain',
 'Name',
 'Opened',
 'Priority',
 'Major incident state',
 'Promoted on',
 'Severity']

In [22]:
INC = INC.rename(columns={'Name':'business_service_name'})

In [23]:
INC['business_service_name'] = INC['business_service_name'].str.strip('[PROD]')
INC['business_service_name'] = INC['business_service_name'].str.strip()

In [24]:
INC_CNT = INC[['Number','business_service_name','State','Priority','Severity']]

In [25]:
INC_CNT = INC_CNT.groupby(['business_service_name','Priority'])['Number'].count().reset_index(name='INC_COUNT')

In [26]:
INC_CNT

,business_service_name,Priority,INC_COUNT
0,360T Trading Platform [Ext],5 - Very low,1
1,3Sixty Coaching and Compliance - NZ,5 - Very low,2
2,ACCT,4 - Low,1
3,AEMforms,5 - Very low,3
4,AFCX - Australian Financial Crime Exchange [Ext],5 - Very low,2
5,AI Machine Learning,5 - Very low,2
6,AI Retail OFI Bank Statements,5 - Very low,4
7,AI Trade Bank Guarantee,5 - Very low,3
8,AI Trade Finance Sanctions Screening,5 - Very low,9
9,AML NetReveal - Global,4 - Low,8


In [27]:
model1map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\model1_mapping.csv')
siam_map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\SIAM_ServiceMapping.csv')

In [28]:
siam_map = siam_map.loc[siam_map.notnull().sum(1).groupby(siam_map.Name).idxmax()]
model1map = model1map.loc[model1map.notnull().sum(1).groupby(model1map.Name).idxmax()]

In [29]:
INC_CNT_model1 = INC_CNT.merge(model1map,how='left',left_on=['business_service_name'],right_on=['Name'])
INC_CNT_SIAM = INC_CNT.merge(siam_map,how='left',left_on=['business_service_name'],right_on=['Name'])

In [30]:
INC_CNT_model1 = INC_CNT_model1[['business_service_name','Priority','INC_COUNT','Domain','Tribe_TechArea','Name']]
INC_CNT_SIAM = INC_CNT_SIAM[['business_service_name','Priority','INC_COUNT','Domain','Tribe_TechArea','Name']]

In [31]:
INC_CNT_model1['Name'] = INC_CNT_model1['Name'].fillna(0)
INC_CNT_SIAM['Name'] = INC_CNT_SIAM['Name'].fillna(0)

In [32]:
INC_CNT_model1 = INC_CNT_model1[INC_CNT_model1['Name'] != 0]
INC_CNT_SIAM = INC_CNT_SIAM[INC_CNT_SIAM['Name'] != 0]

In [33]:
INC_CNT_SIAM.to_csv(r'C:\NotBackedUp\SOE\SIAM\CSV\INC_CNT_PER_APP_WITH_PRIORITY_'+today+'.csv',index=None)
INC_CNT_model1.to_csv(r'C:\NotBackedUp\SOE\Model1\CSV\INC_CNT_PER_APP_WITH_PRIORITY_'+today+'.csv',index=None)

In [34]:
INC_CNT_model1

,business_service_name,Priority,INC_COUNT,Domain,Tribe_TechArea,Name
35,ANZ Shield Credential,4 - Low,4,Australia Retail and Commercial,Digital Customer Experience,ANZ Shield Credential
104,Adobe Experience Manager - Electronic Forms,5 - Very low,2,Australia Retail and Commercial,Digital Customer Experience,Adobe Experience Manager - Electronic Forms
226,CAZ (Commercial A to Z - Desktop) - AU,5 - Very low,3,Australia Retail and Commercial,BANKER EXPERIENCE,CAZ (Commercial A to Z - Desktop) - AU
566,Host Navigator - AU,5 - Very low,7,Australia Retail and Commercial,CUSTOMER CONTACT,Host Navigator - AU
711,Maude - AU,4 - Low,1,Australia Retail and Commercial,CUSTOMER CONTACT,Maude - AU
712,Maude - AU,5 - Very low,9,Australia Retail and Commercial,CUSTOMER CONTACT,Maude - AU
